In [1]:
!pip install mediapipe opencv-python
!pip install ipytone
!pip install pygame
!pip install soundstretch
import subprocess
import IPython.display as ipd
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
import ipytone
import numpy as np
import matplotlib.pyplot as plt
import pygame
import os
import time
!pip install sounddevice
!pip install soundfile
import sounddevice as sd
import soundfile as sf
import urllib.request
import tempfile
import threading
import requests
import io










mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

start_time = None
round_trip_time = None
origin_point = (500, 300)
min_distance = 25 
original_bpm = 75  


last_tempo_change_time = 0
tempo_change_interval = 2  
position = 0
speed = 1.0  
volume = 0.5  


audio_urls = {
    "Option 1": "https://drive.google.com/uc?export=download&id=1dSuG7DN2iCSBdkdwj5qdm_ksk-vWg5qa",
    "Option 2": "https://drive.google.com/uc?export=download&id=1IXkMTfYC-JwiN6p9kIB28trcn1-MvjkC",
    "Option 3": "https://drive.google.com/uc?export=download&id=1Vv2nGpouHiXRGIa3npB08VrwHJr_obPc",
}




def draw_menu(screen):
    """Draw the opening screen with selectable boxes for song choices."""
    screen[:] = (0, 0, 0)  
    cv2.putText(screen, "Select a Song to Start", (150, 100),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.rectangle(screen, (100, 200), (400, 300),
                  (0, 255, 0), -1) 
    cv2.putText(screen, "Mahler Adagietto", (120, 250),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
    cv2.rectangle(screen, (450, 200), (750, 300),
                  (0, 0, 255), -1) 
    cv2.putText(screen, "Mozart Nachtmusik", (470, 250),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
    cv2.rectangle(screen, (250, 350), (550, 450),
                  (255, 255, 0), -1)  
    cv2.putText(screen, "Tchaikovsky Fairy", (270, 400),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
    cv2.imshow('Opening Screen', screen)


    
def check_box_click(x, y):
    """Check which box is clicked based on the mouse coordinates."""
    if 100 <= x <= 400 and 200 <= y <= 300:
        return "Option 1"  
    elif 450 <= x <= 750 and 200 <= y <= 300:
        return "Option 2"  
    elif 250 <= x <= 550 and 350 <= y <= 450:
        return "Option 3"  
    return None  


cap = cv2.VideoCapture(0)
screen = np.zeros((600, 800, 3), dtype=np.uint8)
cv2.namedWindow('Opening Screen')

selected_option = None
song_selected = False
while not song_selected:
    draw_menu(screen)

    def on_mouse_click(event, x, y, flags, param):
        global selected_option, song_selected
        if event == cv2.EVENT_LBUTTONDOWN:
            option = check_box_click(x, y)
            if option:
                selected_option = option
                song_selected = True
                
    cv2.setMouseCallback('Opening Screen', on_mouse_click)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cv2.destroyWindow('Opening Screen')

if song_selected and selected_option:
    try:
        music_url = audio_urls[selected_option]

        response = requests.get(music_url)
        response.raise_for_status()

        data, fs = sf.read(io.BytesIO(response.content), dtype='float32')
        if data.ndim > 1:
            data = data.mean(axis=1) 
        length = len(data)
        position = 0
        speed = 1.0 

        print(f"Loaded {selected_option}: {music_url}")

      
        def audio_callback(outdata, frames, time_info, status):
            global position, speed, data, fs, length, volume
            if status:
                print(status)
           
            indices = position + np.arange(frames) * speed
            indices_int = np.floor(indices).astype(int) % length
            indices_frac = indices - np.floor(indices)
            indices_next = (indices_int + 1) % length
            
            outdata[:, 0] = volume * ((1 - indices_frac) * data[indices_int] + indices_frac * data[indices_next])
            position += frames * speed
            position = position % length

       
        stream = sd.OutputStream(channels=1, callback=audio_callback, samplerate=fs, dtype='float32')
        stream.start()

       
        with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8) as pose:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                
                image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image_rgb.flags.writeable = False
                results = pose.process(image_rgb)

              
                image_rgb.flags.writeable = True
                image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

                
                cv2.circle(image_bgr, origin_point, 10,
                           (0, 0, 255), -1)  

              
                frame_height = frame.shape[0]
                middle_line = frame_height // 2

               
                cv2.line(image_bgr, (0, middle_line), (frame.shape[1], middle_line), (255, 255, 0), 2)

                if results.pose_landmarks:
                 
                    left_wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]
                    x_pos = int(left_wrist.x * frame.shape[1])
                    y_pos = int(left_wrist.y * frame.shape[0])

                    cv2.circle(image_bgr, (x_pos, y_pos), 10,
                               (0, 255, 0), -1) 

                    distance_to_origin = ((x_pos - origin_point[0])**2 +
                                          (y_pos - origin_point[1])**2)**0.5
                    if distance_to_origin < min_distance:
                        current_time = time.time()
                        if start_time is None:
                            start_time = current_time
                        else:
                            round_trip_time = current_time - start_time
                            bpm = 60 / round_trip_time if round_trip_time > 0 else 0
                            print(
                                f"Round trip time: {round_trip_time:.2f} seconds, BPM: {bpm:.2f}")

                            alpha = 0.1 

                            if current_time - last_tempo_change_time >= tempo_change_interval:
                                bpm = 60 / round_trip_time if round_trip_time > 0 else original_bpm
                                target_speed = bpm / original_bpm if bpm > 0 else 1
                                target_speed = max(0.5, min(target_speed, 1.5))
                                speed = (1 - alpha) * speed + alpha * target_speed
                                print(f"Playback speed adjusted to: {speed:.2f}x")
                                last_tempo_change_time = current_time

                            start_time = current_time

                    right_wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]
                    right_wrist_x = int(right_wrist.x * frame.shape[1])
                    right_wrist_y = int(right_wrist.y * frame.shape[0])

                    cv2.circle(image_bgr, (right_wrist_x, right_wrist_y), 10, (255, 0, 0), -1)  # Blue point

                    if right_wrist_y < middle_line:
                        volume = min(1.0, volume + 0.01)  
                    elif right_wrist_y > middle_line:
                        volume = max(0.0, volume - 0.01)
                    print(f"Volume: {volume:.2f}")



                cv2.imshow('Mediapipe Feed', image_bgr)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

        cap.release()
        cv2.destroyAllWindows()
        stream.stop()
        stream.close()

    except Exception as e:
        print(f"Error loading audio: {e}")

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
pygame 2.6.1 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Loaded Option 2: https://drive.google.com/uc?export=download&id=1IXkMTfYC-JwiN6p9kIB28trcn1-MvjkC
